

Team:

Valentina Slisser, 14955792

# Outline

In this notebook we will develop a Markov Network (MN) class and some of the key algorithms for MNs.

An MN combines an _undirected graph_ and a collection of _factors_, and we have coded graphs and factors (specifically, tabular factors) for you. We also provide some of the code for an MN class, and you will complete the rest of it.

This is a high-level outline of the notebook, you will find exercises in most sections.

1. We start with constructing graphs.
2. Next, we construct tabular factors.
3. Then, we design an MN class combining a graph and a collection of tabular factors, this class will support all three fundamental probability queries (joint, marginal and conditional).
4. We then look into reasoning.
5. Finally, we test for independence using the MN structure.

**Table of Exercises**

The exercises and the points they are worth are shown below:

1. Misconception: Structure [0.5]
2. Misconception: Factors [0.5]
3. Misconception: MN [1]
4. Misconception: Joint Distribution [1]
5. Misconception: Reasoning [2]
6. Misconception: Active Paths [1]
7. Misconception: Separation [1]
8. Extended Misconception: MN Structure [1]
9. Extended Misconception: Local Independencies [2]

Do not forget to check our policy on submitting notebooks, and remember to always display results in a coherent and organiser manner.

**Use of AI tools**

In this course we expect _you_ and your team members to author your work.
AI tools are not to be used for drafts, nor code completion, nor revisions, nor as a 'study tool', nor as a source of feedback. If you do use AI, it should not contribute to the substance of what you present as your work.  

At the end of this notebook you will find a section on _Use of AI tools_. **Make sure to read and complete it**.
By submitting a version of this notebook for assessment, you agree with our terms.

# Setting Up

Take care of dependencies:

In [5]:
!pip install tabulate
!pip install git+https://github.com/probabll/pgmini.git

  Cloning https://github.com/probabll/pgmini.git to /tmp/pip-req-build-5nnrv3rl
  Running command git clone --filter=blob:none --quiet https://github.com/probabll/pgmini.git /tmp/pip-req-build-5nnrv3rl
  Resolved https://github.com/probabll/pgmini.git to commit 596a6e5629ff2d97f12c98bca6f3a4747f8a8a8e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [4]:
import pgmini
print(pgmini.__version__)  # this should be 0.2.0 or higher

0.2.0


In [7]:
from collections import defaultdict, deque, OrderedDict
import itertools
from tabulate import tabulate
import numpy as np

# Undirected Graphs

We use a few helpers from [pgmini](https://github.com/probabll/pgmini), we demo here how to use the relevant helpers.

In [8]:
from pgmini.m2 import UGraph

Let's start with the MN _structure_, namely, an **undirected graph**.
Using our _UGraph_ class you can build undirected graphs very easily:


In [9]:
g = UGraph(nodes=['A', 'C', 'B'], edges=[('B', 'A'), ('C', 'A'), ('B', 'C')])
print(g)

+---------+
| edges   |
+=========+
| A -- B  |
+---------+
| A -- C  |
+---------+
| B -- C  |
+---------+


In [10]:
g.nodes

frozenset({'A', 'B', 'C'})

In [11]:
g.edges

frozenset({('A', 'B'), ('A', 'C'), ('B', 'C')})

See that in an undirected graph a node has neighbours, rather than parents and children.

In [12]:
g.neighbors['A'], g.neighbors['B'], g.neighbors['C']

(frozenset({'B', 'C'}), frozenset({'A', 'C'}), frozenset({'A', 'B'}))

# Factors

In [13]:
from pgmini.m1 import OutcomeSpace
from pgmini.m2 import TabularFactor

Next, we need _factors_, in order to parameterise distributions over the MN structure.
A factor is assigns joint affinity to the outcomes of a collection of random variables (rvs) called the factor's _scope_. In this course, affinities are always non-negative (that is, >= 0).

Consider two binary rvs, namely $A$ and $B$. A factor whose scope is the set or rvs $\{A, B\}$ maps joint outcomes in $\operatorname{Val}(A, B)$ to $\mathbb R_{\ge 0}$.

There are many ways to represent factors in a computer, our strategy for now is to use a **table**. That's okay because for now we are only interested in discrete rvs (with countably finite sample spaces) and not too many of them will interact at once.

A **tabular factor** is essentially a table whose rows list joint outcomes and their corresponding affinities. In the example above, the table would have 4 rows (one for each joint outcome of $A, B$) and 3 columns (on for rv in the factor's scope and one more for the factor value).

The table below illustrates the basic datastructure.

|   $ A $    |   $ B $     |   $ \phi(A, B) $        |
| :--------: | :---------: | :---------------------: |
|   $ a^0 $  | $ b^0 $     | $ \phi(a^0, b^0) $  |
|   $ a^0 $  | $ b^1 $     | $ \phi(a^0, b^1) $  |
|   $ a^1 $  | $ b^0 $     | $ \phi(a^1, b^0) $  |
|   $ a^1 $  | $ b^1 $     | $ \phi(a^1, b^1) $  |

Internally we use a _tensor_, rather than a table, each axis of the tensor is associated with the outcome space of one of the rvs in the scope, in order. The helper class OutcomeSpace helps us then map the outcome name of a given rv (for example $b^1$ for rv $B$) to a 0-based integer used to index a coordinate of the axis associated with $B$ (the second axis, in this example).

In [14]:
spacesABC = {'A': OutcomeSpace(['a0', 'a1']), 'B': OutcomeSpace(['b0', 'b1']), 'C': OutcomeSpace(['c0', 'c1'])}

In [15]:
# Define factors
fAB = TabularFactor(
    ['A', 'B'],
    spacesABC,  # irrelevant outcome spaces are ignored
    [[1, 10], [5, 5]]
)
print(fAB)

A    B      Value
---  ---  -------
a0   b0         1
a0   b1        10
a1   b0         5
a1   b1         5


In [16]:
# Define factors
fBC = TabularFactor(
    ['B', 'C'],
    spacesABC,
    [[2, 2], [1, 6]]
)
print(fBC)

B    C      Value
---  ---  -------
b0   c0         2
b0   c1         2
b1   c0         1
b1   c1         6


In [17]:
# Define factors
fCA = TabularFactor(
    ['C', 'A'],
    spacesABC,
    [[1, 3], [2, 1]]
)
print(fCA)

C    A      Value
---  ---  -------
c0   a0         1
c0   a1         3
c1   a0         2
c1   a1         1


You can **evaluate** a factor by assigning all of the rvs in its scope:

In [18]:
fCA.evaluate({'A': 'a0', 'B': 'b1', 'C': 'c1'})  # irrelevant assignments are ignored

np.float64(2.0)

To stress this point, you cannot _evalute_ a factor unless you assign _all of the rvs in its scope_:

In [19]:
try:
    fCA.evaluate({'A': 'a0'})  # but rvs in scope must be assigned
except KeyError:
    print("You cannot _evalute_ a factor unless you assign all of the rvs in its scope.")
    print("Maybe you meant to reduce the factor instead?")

You cannot _evalute_ a factor unless you assign all of the rvs in its scope.
Maybe you meant to reduce the factor instead?


In some situations however you do intend to assign only a subset of the rvs in the factor's scope, but in those situations then what you are looking for is the **reduce** operation.
While evaluate produces a non-negative real number, reduce produces another tabular factor:

In [20]:
fCA.reduce({'A': 'a0'})

C      Value
---  -------
c0         1
c1         2

In [21]:
fCA.reduce({'C': 'c1', 'B': 'b0'})  # and, again, irrelevant assignments are ignored

A      Value
---  -------
a0         2
a1         1

Watch out! If you reduce all variables in the scope of a factor, what you get is still a factor object:

In [22]:
fCA.reduce({'A': 'a0', 'C': 'c1'})

  Value
-------
      2

If you want to access the scalar that's left after reducing everything as a simple python float, you can evaluate the reduced factor with an empty assignment or access the underlying tensor `values`:

In [23]:
# like this
print(fCA.reduce({'A': 'a0', 'C': 'c1'}).evaluate(dict()))
# or like this
print(fCA.reduce({'A': 'a0', 'C': 'c1'}).values.item())

2.0
2.0


You can also take the **product** of factors. For example, see how the product of fAB and fBC will match the common rvs in their scope and return the correct factor:

In [24]:
fAB.product(fBC)

A    B    C      Value
---  ---  ---  -------
a0   b0   c0         2
a0   b0   c1         2
a0   b1   c0        10
a0   b1   c1        60
a1   b0   c0        10
a1   b0   c1        10
a1   b1   c0         5
a1   b1   c1        30

There's a nice little python trick that is very useful. You can use `functools.reduce` to apply a certain binary operator repeatedly (don't confuse `functools.reduce` with factor reduction, these are unrelated operations). See how we can use it to take the product of a whole collection of factors:

In [25]:
import functools

In [26]:
fABC1 = functools.reduce(TabularFactor.product, [fAB, fBC, fCA])
fABC1

A    B    C      Value
---  ---  ---  -------
a0   b0   c0         2
a0   b0   c1         4
a0   b1   c0        10
a0   b1   c1       120
a1   b0   c0        30
a1   b0   c1        10
a1   b1   c0        15
a1   b1   c1        30

The order in which you take the product does not matter for `evaluate` but the internal tensor representation of the factor may be different, with axes permuted, that's because **product** does not sort the combined scope. This is not at all a problem, we are only remarking this so you don't get confused.

In [27]:
fABC2 = functools.reduce(TabularFactor.product, [fCA, fBC, fAB])
fABC2

C    A    B      Value
---  ---  ---  -------
c0   a0   b0         2
c0   a0   b1        10
c0   a1   b0        30
c0   a1   b1        15
c1   a0   b0         4
c1   a0   b1       120
c1   a1   b0        10
c1   a1   b1        30

You can validate our claim above by inspection, but we can also show it to you programmatically. This way we also teach you how to use a few more things : )

In [28]:
from pgmini.m1 import enumerate_joint_assignments

In [29]:
for assignment in enumerate_joint_assignments(['A', 'B', 'C'], spacesABC):
    assert np.isclose(fABC1.evaluate(assignment), fABC2.evaluate(assignment))

There are two other operations on factors that are useful, but which need to be used with great care: normalisation and marginalisation.

While the result of applying **reduce** or **product** is always consistent with the MN structure, that is not necessarily true for **marginalize** and **normalize**. The reason is that if you apply these operations to factors that do not yet incorporate all the necessary dependencies, what you get 'looks' like a marginal or a probability distribution in terms of form, but the objects have nothing to do with the MN structure you are working with.

As a rule of thumb, at least until we cover some of the results of Chapter 9, you should only marginalise a factor that already includes all rvs, and you should only normalise a factor that either includes all rvs or whatever rv is missing is only missing because you marginalised them.

For example, in our little A -- B -- C -- A network, if you marginalise A from fAB and normalise the result, it will look like you have a marginal probability distribution over A. But that distribution cannot possibly belong to  this MN, after all, we completely ignored the other factors in order to produce this result.  

In [30]:
# this LOOKS LIKE a marginal distribution,
# but it is not the marginal of A in this MN
fAB.marginalize({'B'}).normalize()

A      Value
---  -------
a0   0.52381
a1   0.47619

In [31]:
# compare the wrong one (above) to the correct one (below)
fABC1.marginalize({'B', 'C'}).normalize()

A       Value
---  --------
a0   0.615385
a1   0.384615

# The Misconception Example

We are ready to code the graph structure of the Misconception MN discussed in class (Figure 1).

**EXERCISE - Misconception: Structure.** Construct the MN structure of the Misconception example (Figure 1 in class) using _UGraph_ and display it.

In [33]:

from pgmini.m2 import UGraph

misconception_graph = UGraph(
    nodes=['A', 'B', 'C', 'D'],
    edges=[
        ('A', 'B'),
        ('A', 'C'),
        ('B', 'D'),
        ('C', 'D')
    ]
)

print(misconception_graph)

+---------+
| edges   |
+=========+
| A -- B  |
+---------+
| A -- C  |
+---------+
| B -- D  |
+---------+
| C -- D  |
+---------+


**EXERCISE - Misconception: Factors.** Construct _TabularFactor_ objects for all factors in the Misconception example discussed in class (use the same numerical values as we did in class; Figure 2) and display them.

In [36]:
from pgmini.m1 import OutcomeSpace
from pgmini.m2 import TabularFactor

spacesABCD = {
    'A': OutcomeSpace(['a0', 'a1']),
    'B': OutcomeSpace(['b0', 'b1']),
    'C': OutcomeSpace(['c0', 'c1']),
    'D': OutcomeSpace(['d0', 'd1'])
}

phi_D_A = TabularFactor(
    ['D', 'A'],
    spacesABCD,
    [[100, 1], [1, 100]]
)

phi_A_B = TabularFactor(
    ['A', 'B'],
    spacesABCD,
    [[30, 5], [1, 10]]
)

phi_C_D = TabularFactor(
    ['C', 'D'],
    spacesABCD,
    [[1, 100], [100, 1]]
)

phi_B_C = TabularFactor(
    ['B', 'C'],
    spacesABCD,
    [[100, 1], [1, 100]]
)

print("Factor φ(D,A):")
print(phi_D_A)
print("\nFactor φ(A,B):")
print(phi_A_B)
print("\nFactor φ(C,D):")
print(phi_C_D)
print("\nFactor φ(B,C):")
print(phi_B_C)

Factor φ(D,A):
D    A      Value
---  ---  -------
d0   a0       100
d0   a1         1
d1   a0         1
d1   a1       100

Factor φ(A,B):
A    B      Value
---  ---  -------
a0   b0        30
a0   b1         5
a1   b0         1
a1   b1        10

Factor φ(C,D):
C    D      Value
---  ---  -------
c0   d0         1
c0   d1       100
c1   d0       100
c1   d1         1

Factor φ(B,C):
B    C      Value
---  ---  -------
b0   c0       100
b0   c1         1
b1   c0         1
b1   c1       100


In [35]:
# # some tests (assumes you named the factors phi1, ..., phi4 as we did in class)
# example_obs1 = {'A': 'a0', 'B': 'b1', 'C': 'c0', 'D': 'd1'}
# assert phi1.evaluate(example_obs1) == 5.
# assert phi2.evaluate(example_obs1) == 1.
# assert phi3.evaluate(example_obs1) == 100.
# assert phi4.evaluate(example_obs1) == 1.

# example_obs2 = {'A': 'a0', 'B': 'b1', 'C': 'c1', 'D': 'd0'}
# assert phi1.evaluate(example_obs2) == 5.
# assert phi2.evaluate(example_obs2) == 100.
# assert phi3.evaluate(example_obs2) == 100.
# assert phi4.evaluate(example_obs2) == 100.

# Markov Network

Then, our **Markov Network** (MN) data structure will store a UGraph for the BN structure and a collection of TabularFactor objects.

In [37]:
from pgmini.m2 import UGraph, TabularFactor
from pgmini.m1 import OutcomeSpace

def build_graph_from_factors(factors):
    """
    Build an undirected graph from a list of factors by connecting variables
    that appear together in any factor.

    Args:
        factors (list): List of TabularFactor objects

    Returns:
        UGraph: An undirected graph representing the Markov Network structure
    """
    # Collect all unique nodes (variables) from the factors
    nodes = set()
    edges = set()

    for factor in factors:
        # Add all variables in the factor's scope to nodes
        factor_nodes = list(factor.scope)
        nodes.update(factor_nodes)

        # Create edges between all pairs of nodes in the factor's scope
        for i in range(len(factor_nodes)):
            for j in range(i+1, len(factor_nodes)):
                edges.add(tuple(sorted([factor_nodes[i], factor_nodes[j]])))

    # Create and return the UGraph
    return UGraph(nodes=list(nodes), edges=list(edges))

# Define outcome spaces
spacesABCD = {
    'A': OutcomeSpace(['a0', 'a1']),
    'B': OutcomeSpace(['b0', 'b1']),
    'C': OutcomeSpace(['c0', 'c1']),
    'D': OutcomeSpace(['d0', 'd1'])
}

# Create factors (using values from the class slides)
# φ(D,A)
phi_D_A = TabularFactor(
    ['D', 'A'],
    spacesABCD,
    [[100, 1], [1, 100]]
)

# φ(A,B)
phi_A_B = TabularFactor(
    ['A', 'B'],
    spacesABCD,
    [[30, 5], [1, 10]]
)

# φ(C,D)
phi_C_D = TabularFactor(
    ['C', 'D'],
    spacesABCD,
    [[1, 100], [100, 1]]
)

# φ(B,C)
phi_B_C = TabularFactor(
    ['B', 'C'],
    spacesABCD,
    [[100, 1], [1, 100]]
)

# Create the list of factors
factors = [phi_D_A, phi_A_B, phi_C_D, phi_B_C]

In [38]:
class MarkovNetwork:
    """
    An MN combines a UGraph (we use an implementation from pgmini.m2)
     and a collection of Factors (we use TabularCPD from pgmini.m2)
    """

    def __init__(self, factors: list):
        """
        factors: a list of Factor objects

        We build the MN structure from the list of factors to avoid a situation where the
        two are not coherent with one another.
        Building it is part of an exercise, read on and you will find out more.
        """
        self.graph = build_graph_from_factors(factors)
        self.factors = list(factors)
        self._normalizer = None  # this is a placeholder, there's a method that computes this later

    def joint_unnormalized_probability(self, assignment: dict) -> float:
        """
        Compute and return the joint unnormalised probability (float) of an assignment of the rvs.
            This assignment is regarded as 'complete' (all rvs have been assigned).

            If you ever have to query an assignment that is incomplete,
            you need to think about the _kind_ of query you are making and find a better method in this class.

        assignment: a dict mapping each rv to an outcome in the rv's outcome space
        """
        # **YOUR SOLUTION HERE**
        raise NotImplementedError("Implementing this is an exercise")

    def joint_probability(self, assignment: dict) -> float:
        """The actual joint probability of the assignment (that is, after normalisation)"""
        tilde_p = self.joint_unnormalized_probability(assignment)
        return tilde_p / self.normalizer()

    def normalizer(self) -> float:
        """Compute and return the normalizer"""
        if self._normalizer is None:
            tilde_P = functools.reduce(TabularFactor.product, self.factors)
            self._normalizer = tilde_P.values.sum()
        return self._normalizer

    def marginal_probability(self, query_assignment=dict()) -> float:
        """
        Compute P(query_assignment) using factor operations.
        """

        # shorter names
        q = query_assignment
        # some rvs are not assigned, we are going to have to marginalise them
        unassigned_rvs = self.graph.nodes - q.keys()

        # for now, we have no better strategy than to compute the entire joint distribution
        # later in the course, we will have better algorithms for this

        # Steps:
        # 1. construct the complete tilde_P factor by taking the product of all factors
        # 2. marginalise out the unassigned rvs
        # 3. normalise the factor to get a marginal distribution over the query rvs
        marginal_dist = functools.reduce(TabularFactor.product, self.factors).marginalize(unassigned_rvs).normalize()

        # we can simply evaluate the new factor using the query assignment
        return marginal_dist.evaluate(q)

    def is_factor_relevant(self, factor, assignment):
        """Helper function to determine whether an assignment is relevant to a given factor"""
        for rv in factor:
            if rv in assignment:
                return True
        return False

    def conditional_probability(self, query_assignment: dict, evidence_assignment: dict) -> float:
        """
        Compute P(query | evidence) using any method you like
         Of course, you can use any functionality available in this class.
        """
        # **YOUR SOLUTION HERE**
        raise NotImplementedError("Implementing this is an exercise in the Reasoning section")


**EXERCISE - Misconception: MN.** Construct the a _MarkovNetwork_ object for the Misconception example (i.e., the graph from Figure 1 and factors from Figure 2) and display its structure.

To make sure that there's no mismatch between the graph and the collection of factors, our class expects a list of factors and it builds the graph implied by it. You should not modify the constructor of _MarkovNetwork_ to build the graph, rather, you should implement the function `build_graph_from_factors` above. You will know you implemented it correctly if you obtain the Misconception structure from the list of factors.

In [ ]:
# **YOUR SOLUTION HERE**
misconception_mn = None

In [ ]:
# assert misconception_mn.joint_unnormalized_probability({'A': 'a0', 'B': 'b0', 'C': 'c0', 'D': 'd0'}) == 300000.0
# assert misconception_mn.joint_unnormalized_probability({'A': 'a0', 'B': 'b1', 'C': 'c1', 'D': 'd0'}) == 5000000.0

**EXERCISE - Misconception: Joint Distribution.** Complete the `joint_unnormalized_probability` method of the `MarkovNetwork` class, then for the Misconception MN, display the complete joint distribution in a table and verify that its normaliser is correct.

Here's how you should format the table (you can use `tabulate` or your preferred helper for visualising tables):
* one joint outcome per row
* columns for each rv and two extra columns at the end, one for the unnormalised probability and one for the normalised probability
* for unnormalised probability, use 2 decimals of precision
* for probability, use 6 decimals of precision
* for the order of the joint outcomes, use `('A', 'B', 'C', 'D')`

This should reconstruct Table 1 from the class slides.

In [ ]:
# **YOUR SOLUTION HERE**

In [ ]:
# assert np.isclose(misconception_mn.normalizer(), 7201840.0)

# Reasoning

We can use probability calculus to appreciate the effect of observing a variable may or may not have on our beliefs about other variables.

**EXERCISE - Misconception: Reasoning.** Make the relevant queries to the distribution of the Misconception example to compute the probability that C has the misconception in the following cases, and discuss your observations:

1. nothing else is known;
2. given that A has the misconception;
3. given that B has the misconception;
4. given that D has the misconception;
5. given that B and D have the misconception;
6. given that A, B and C have the misconception.


For the discussion you can make a remark about each of the 6 cases, your remark does not have to be extensive, but it should try to explain why the probability goes up or down relative to some other case of interest. For the first case, you can remark on whether or not you could have guessed the low/high value from local factor values involving C.


Tip for implementation: We have implemented `marginal_probability` for you, but you will also need `conditional_probability` for this exercise (which you should implement yourself). There are various implementations that are possible, some more efficient than others, we are not going to grade on efficiency (but will provide feedback). You can create test cases for your implementation by computing some examples from first principles (probability calculus) using Table 1, as we did in class.

As always, make sure your solution is organised, your TA cannot grade it if they cannot understand it.

In [ ]:
# **YOUR SOLUTION HERE**

# Separation

Separation is a tool for ascertaining (conditional) independence statements based solely on the MN structure (that is, knowing the graph is enough, we do not need a complete distribution with its underlying local probability models).

Separation depends on the concept of a path, which may be active (enable influence) or inactive (block influence), as discussed in class. When we make a sep claim we are making a claim that holds for _all_ paths between the relevant nodes.

In general, in an undirected graph, enumerating all path is an exponential-time algorithm, so implementing spearation efficiently requires careful algorithms.

A polynomial-time algorithm for separation is possible and not too complex, but for didactic purposes, here we will be _enumerating_ paths exhaustively and one by one (for this notebook this is okay because we only have small MNs).

You will first implement the test to determine whether a path is active or not. After that, you will implement separation.

**EXERCISE - Misconception: Active Paths.** Implement a helper function to decide whether a path is active in a _UGraph_.

Then use `make_all_paths_table` below to print the paths between all pairs of nodes in the Misconception MN and the paths's status (active or not) before and after observing an evidence set. Print a table with `evidence={'B'}` and one with `evidence={'B', 'D'}`.

In [ ]:
def is_path_active(graph: UGraph, path: list, evidence=set()):
    """
    Return True if a given path (list of nodes) is active given evidence.
    """
    raise NotImplementedError("Implementing this is an exercise!")

In [ ]:
def make_all_paths_table(graph: UGraph, evidence=set()):
    paths_table = []
    for x, y in itertools.combinations(graph.nodes, 2):
        for path in graph.enumerate_paths(x, y):
            paths_table.append([x, y, " -- ".join(path), is_path_active(graph, path), is_path_active(graph, path, evidence)])
    return tabulate(paths_table, headers=['From', 'To', 'Path', 'Active without evidence', f"Active given {', '.join(evidence)} "])

In [ ]:
# **YOUR SOLUTION HERE**

**EXERCISE - Misconception: Separation.** Implement seperation by enumeration of paths. Test it for the following 4 statements:

1. $A \perp C$
2. $A \perp C \mid B$
2. $A \perp C \mid B, D$
5. $B \perp D \mid A, C$


In [ ]:
def sep(mn_structure: UGraph, X: set(), Y: set, Z=set()):
    """
    Return True if d-sep(X; Y|Z) or False otherwise.
    """
    raise NotImplementedError("Implementing this is an exercise")

In [ ]:
# **YOUR SOLUTION HERE**

**EXERCISE - Extended Misconception: MN Structure.** Construt the MN structure for the following extension to the Misconception example and display it.

_We now have two different classes. In class 1, we have A, B, C, D. In class 2, we have C, E, F, G. Except for C, who is taking both classes, students don't communicate across classes. As before, in class 1, A is not on speaking terms with C, and, similarly, B and D are not on speaking terms either. In class 2, C and F are not on speaking terms, and, similarly, G and E are not on speaking terms either._

In [ ]:
# **YOUR SOLUTION HERE**
extended_misconception = None

**EXERCISE - Extended Misconception: Local Independencies.** This exercise concerns the _Extended Misconception MN Structure_. Using sep as implemented above, show for each node $X_i$ in the MN structure that, _given its Markov Blanket_, $X_i$ is independent of all other nodes in the structure. For each node, also show that if we are not given all of the nodes in the Markov blanket (for example, ommit one of them), the separation claim reverses.

As always, you need to actually display the results in a coherent and organiser manner. If the TA cannot visualise or understand what you did, they might not be able to grade your solution.

In [ ]:
# **YOUR SOLUTION HERE**

_FYI_ `pgmini.m2.graph` ships an efficient algorithm for _separation_, in case you want to learn about it.

# Use of AI Tools

By submitting this notebook for grading you testify that:

* AI did not draft an earlier version of your work.
* You did not use AI-powered code completion.
* You did not implement algorithms suggested by an AI tool.
* AI did not revise a version of your work.
* You did not implement suggestions made by an AI tool.


_You_ in the sentences above refers to you and all your team members.
_AI_ refers to LM-based tools and assistants (e.g., ChatGPT, Gemini, UvA AI chat, etc.).

If you did make use of an AI tool, you should describe the uses you made of it below. Or indicate that no such tool was used.

**TYPE YOUR STATEMENT HERE**